# Warning

After pulling 2017.2.dev from upstream (sigh), things are acting weirdly. Again.

* `plot()` crashes if called with a function having NaNs. Why would I do that? Because:
* `poisson()`, which naively uses an Hermite `FunctionSpace` and applies a standard `DirichletBC` produces such a solution (?)
* The strange inner effects for the (correct) Hermite solutions have now disappeared from the plots. The cross sections look good. So either some interpolation is being done when calling `u(x)` on a solution `u` and it wasn't before, or something mysterious is going on.
* The relative differences between Lagrange and Hermite solutions both in $L^2$ and $H^1$ norm have exploded. The latter is well above 1000%, yes: *ten times*.

WTF?

# Poisson 2D

This notebook solves the linear problem:

$$ - \Delta u = f \text{ in } \Omega = [0,1]^2$$ 

with

$$ f(x, y) = 10 \exp^{-50 [(x - 1/2)^2 + (y - 1/2)^2)]} $$

and boundary conditions given by

$$    u(x, y) = 0 \text{ for } x = 0 \text{ or } x = 1 $$

and

$$ \frac{\partial u}{\partial n}(x, y) = sin(5 x) \text{ for } y = 0 \text{ or } y = 1. $$

The purpose is to find what the issues are while setting BCs using Hermite elements and fix them. See [Fixing the boundary conditions](#Fixing-the-boundary-conditions).

In [ ]:
from __future__ import print_function
from dolfin import *
#import matplotlib as mpl
#mpl.rcParams['figure.figsize'] = (12,12)  # This doesn't work
import matplotlib.pyplot as pl
import numpy as np
from IPython.display import Math
import autograd.numpy as np
import nbimporter
from interpolation import *
from boundary import apply_dirichlet_hermite, plot_hermite_dofs
from utils import make_derivatives

%matplotlib inline

parameters["form_compiler"]["cpp_optimize"] = False
parameters["form_compiler"]["optimize"] = False
parameters['form_compiler']['representation'] = 'quadrature'

In [ ]:
def poisson(V):
    """ Solves Poisson's eq on the unit square.
    This code is based on DOLFIN's homonymous demo by Anders Logg.
    
    WARNING: this won't work for Hermite elements unless DirichletBC
             has been fixed.

    Arguments:
    ----------
        V: FunctionSpace.
    Returns:
    --------
        Solution (Function in V).
    """
    assert isinstance(V, FunctionSpace), "Duh."

    @make_derivatives
    def rhs(x,y):
        return 10*np.exp(-50*((x - 0.5)**2 + (y - 0.5)**2))

    @make_derivatives
    def neu(x,y):
        return np.sin(5*x)
    
    def boundary(x):
        return near(x[0], 0) or near(x[0], 1)

    deg = V.ufl_element().degree()
    # Need to project to work around lack of evaluate_dofs() for Hermite
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, boundary)

    u = TrialFunction(V)
    v = TestFunction(V)
    a = inner(grad(u), grad(v))*dx
    
#    if V.ufl_element().family().lower() == 'hermite':
#        f = interpolate_hermite(rhs, V)
#        g = interpolate_hermite(neu, V)
#    else:
#        f = Expression("10*exp(-50*(pow(x[0]-0.5, 2) + pow(x[1]-0.5, 2)))",
#                       degree=V.ufl_element().degree())
#        g = Expression("sin(5*x[0])", degree=2)
    f = Expression("10*exp(-50*(pow(x[0]-0.5, 2) + pow(x[1]-0.5, 2)))",
                   degree=deg)
    g = Expression("sin(5*x[0])", degree=deg)
    if V.ufl_element().family().lower() == 'hermite':
        f = project(f, V)
        g = project(g, V)

    L = f*v*dx + g*v*ds
    u = Function(V)
    solve(a == L, u, bc)

    return u

## Four (not so) different solutions

We use P1, P2 and P3 Lagrange elements as well as P3 Hermite elements and compare the results.

In [ ]:
pl.figure(figsize=(12, 12))

# Solve 3 times using different Lagrange elements of different degrees 
mesh = UnitSquareMesh(20, 20, 'crossed')
W = [None] + [FunctionSpace(mesh, "Lagrange", i) for i in range(1, 4)]
lag = [None] + [poisson(W[p]) for p in range(1, 4)]

for i in range(1, 4):
    pl.subplot(2, 2, i)
    plot(lag[i], title = "P%d Lagrange" % i, cmap='bone')
    
# Solve using Hermite elements
V = FunctionSpace(mesh, "Hermite", 3)
her = poisson(V)
pl.subplot(2,2,4)
_ = plot(her, title="P3 Hermite", cmap='bone')

## Relative errors

Depending on how we bring both solutions to a common function space and which norm we choose, we find very different errors. For the simple case where we only compare values at the vertices (i.e. we work CG1) it will be around 10% measured in the $H^1$ norm and 4% in the $L^2$ norm.

Here we would like to observe the fact that orthogonal projection of a function $u$ to obtain its discretisation $u_h$ yields a better approximation than nodal interpolation: In the first two series of plots we simply evaluate the functions at the vertices, and in the third we use dolfin's `project()`. However the results are a bit confusing...

In [ ]:
dif = None # HACK
def plots(what=1):
    """ Blah. """
    global dif

    pl.figure(figsize=(12,4))
    eqs = ''#\\begin{align}'

    for i in range(1,4):
        if what == 1:  # Consider only vertex values and manually compute norm
            lv = lag[i].compute_vertex_values()
            hv = her.compute_vertex_values()
            difv =  lv - hv
            numvert = int(np.sqrt(mesh.num_vertices()))
            dif = difv.reshape((numvert, numvert))
            eqs += '\\text{Norms of differences at vertices: }\ \ \ '
            eqs += '\\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{L_2}}{||u_{\\text{l%d}}||_{L_2}} \\approx %.4f,\ \ \\\~ '\
                   % (i, i, np.linalg.norm(difv, ord=2) / np.linalg.norm(lv, ord=2))
            pl.subplot(1,3,i)
            pl.imshow(dif, cmap='bone')
        elif what == 2:  # Manually interpolate into W[1] (CG1)
            difv = lag[i].compute_vertex_values() - her.compute_vertex_values()
            dif = Function(W[1])
            dif.vector().set_local(difv[dof_to_vertex_map(W[1])])
            eqs += '\\text{Manual interpolation into CG1: }\ \ \ '
            eqs += '\\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{L_2}}{||u_{\\text{l%d}}||_{L_2}} \\approx %.4f,\ \ \ '\
                   ' \\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{H_1}}{||u_{\\text{l%d}}||_{H_1}} \\approx %.4f \\\~ '\
                   % (i, i, norm(dif, 'L2') / norm(lag[i], 'L2'), 
                      i, i, norm(dif, 'H1') / norm(lag[i], 'H1'))
            pl.subplot(1,3,i)
            plot(dif, title="|Lag%d - Her3|" % i, cmap='bone')
        elif what == 3:  # Project onto each of the W[i]
            proj = project(lag[i] - her, W[i])
            eqs += '\\text{After projection onto CG%d: }\ \ \ ' % i
            eqs += '\\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{L_2}}{||u_{\\text{l%d}}||_{L_2}} \\approx %.4f,\ \ \ '\
                   ' \\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{H_1}}{||u_{\\text{l%d}}||_{H_1}} \\approx %.4f \\\~ '\
                   % (i, i, norm(dif, 'L2') / norm(lag[i], 'L2'), i, i, norm(dif, 'H1') / norm(lag[i], 'H1'))
            pl.subplot(1,3,i)
            plot(dif, title="|Lag%d - Her3|" % i, cmap='bone')

    eqs += ''#'\\end{align}'
    return Math(eqs)

The first check is to compute the norm of the difference of the values of all solutions at the vertices.

In [ ]:
plots(1)

The following should produce exactly the same result. We take the difference of one Lagrange and the Hermite solution and use this to populate the `vector()` of a `Function` on `CG1`. Then we use `dolfin.norm()` to compute both $L^2$ and $H^1$ norms, but the results differ from the previous ones. **What's going on?**

In [ ]:
plots(2)

Finally we use `dolfin.project()` to project the Hermite solution onto each of the Lagrange spaces and compute the norms there, again with `dolfin.norm()`.

In [ ]:
plots(3)

A final plot with some "cross-sections" at fixed ordinates for the P3 Lagrange vs Hermite elements. We need to cap the relative error because of very small denominators.

In [ ]:
xx = np.linspace(0,1,100)
pl.figure(figsize=(12,8))
error_threshold = 0.5

for y in np.linspace(0, 1, 4):
    pl.plot(xx, [min(error_threshold, np.abs(dif(x,y)/lag[3](x,y))) for x in xx],
            label="y = %.1f" % y)
    #pl.plot(xx, [dif(x,y) for x in xx], label="D@%.1f" % y)
    #pl.plot(xx, [her([x,y]) for x in xx], label="H@%.1f" % y)
    #pl.plot(xx, [lag([x,y]) for x in xx], label="L@%.1f" % y)
pl.title("Relative error (capped at %.1f) at multiple ordinates" % error_threshold)
_ = pl.legend()

# Fixing the boundary conditions

We try to use the method developed in [boundary.ipynb](boundary.ipynb) for [Poisson1D](Poisson1D.ipynb) in higher dimensions. However, issues other than rows corresponding to boundary elements seem to enter into play. In particular, it might be that some *inner* Hermite dofs are being set to 0 by mistake. See [Interior effects](#Interior-effects).

We now simplify the problem:

$$ - \Delta u = f \text{ in } \Omega = [0,1]^2$$ 

with

$$ f(x, y) = 4 $$

and boundary conditions given by

$$    u(x, y) = 2 \text{ on } \partial \Omega. $$

In [ ]:
from dolfin import *
import matplotlib.pyplot as pl
%matplotlib inline
import autograd.numpy as np
import nbimporter
from boundary import apply_dirichlet_hermite, plot_hermite_dofs
from dofs import plot_dofs, list_bubble_dofs
from interpolation import *
from utils import make_derivatives

@make_derivatives
def rhs(x,y):
    return 10*np.exp(-50*((x[0] - 0.5)**2 + (x[1] - 0.5)**2))

In [ ]:
from utils import fnand, fnor

mesh = UnitSquareMesh(20, 20, 'crossed')
V = FunctionSpace(mesh, 'Hermite', 3)

top = lambda x: near(x[1], 1.)
right = lambda x: near(x[0], 1.)
bottom = lambda x: near(x[1], 0.)
left = lambda x: near(x[0], 0.)

u0 = make_constant(2.0, V)
bc = DirichletBC(V, u0, fnor(top, right, bottom, left))

class td_subdomain(SubDomain):
    """ Top and bottom sides of the domain. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return near(x[1], 0.) or near(x[1], 1.)

class lr_subdomain(SubDomain):
    """ Left and right sides of the domain. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return near(x[0], 0.) or near(x[0], 1.)

# We will use this to mark boundaries for natural boundary conditions
exterior_facet_domains = FacetFunction("uint", mesh, value=1)
#exterior_facet_domains.set_all(1)

natural_boundary = td_subdomain()
natural_boundary.mark(exterior_facet_domains, 0)
ds_neu = ds(subdomain_data = exterior_facet_domains, subdomain_id=0)

u = TrialFunction(V)
v = TestFunction(V)
#g = make_constant(0.0, V)
f = make_constant(4.0, V)
a = inner(grad(u), grad(v))*dx
F = f*v*dx # + g*v*ds_neu

# Compute solution
A = assemble(a)
b = assemble(F)
Ac = A.copy()
print("Applying BCs... ", end='')
apply_dirichlet_hermite(A, b, bc)
print("done.")
her = Function(V)
H = her.vector()
_ = solve(A, H, b)

We first make sure that the routines setting the BCs know what nodes in the boundary to look at. `plot_hermite_dofs()` draws all nodes for a boundary condition which have Hermite dofs associated to them:

In [ ]:
pl.figure(figsize=(10,10))
plot_hermite_dofs(bc)

We can also display all Hermite dofs over facets, for no particular reason. ;)

In [ ]:
bbdofs = list_bubble_dofs(V)
pl.figure(figsize=(10,10))
plot_dofs(V, bbdofs)
_ = pl.title("The %d bubble dofs" % len(bbdofs))

The solution to the problem above looks almost correct:

In [ ]:
pl.figure(figsize=(10,10))
pl.xlim((-0.1,1.1))
pl.ylim((-0.1,1.1))
plot(her, cmap='bone')
_ = pl.title("P3 Hermite")

## Interior effects

Even thought the solution seems correct, there are some strange effects. Notice the wiggly cross-sections in the following plot: might it be that (some of?) the Hermite dofs *inside* the domain are being set to zero?

In [ ]:
pl.figure(figsize=(12,8))
xx = np.linspace(0, 1, 100)
for y in np.linspace(0.1,0.9,5):
    pl.plot(xx, [her([x, y]) for x in xx], label="$y=%.1f$" % y)
_ = pl.legend(fancybox=True, framealpha=0.8)

Are Dirichlet BCs being enforced?

In [ ]:
u0a = u0.vector().array()
ha = her.vector().array()
bdofs = list(filter(lambda x: x is not None,
                    [k if v == 2.0 else None for k,v in bc.get_boundary_values().items()]))

np.allclose(her.vector().array()[bdofs], 2.0)

## Solution with P2 Lagrange elements

As a sanity check, here is one solution known to be correct:

In [ ]:
W = FunctionSpace(mesh, "Lagrange", 2)
u0 = Constant(2.0)
bc = DirichletBC(W, u0, fnor(top, right, bottom, left))

# Define variational problem
u = TrialFunction(W)
v = TestFunction(W)
f = Constant(4.)
#g = Constant(0.)
a = inner(grad(u), grad(v))*dx
F = f*v*dx #+ g*v*ds

# Compute solution
lag = Function(W)
solve(a == F, lag, bc)

In [ ]:
pl.figure(figsize=(10,10))
pl.xlim((-0.1, 1.1))
pl.ylim((-0.1, 1.1))
plot(lag, cmap='bone')
_ = pl.title("P2 Lagrange")

The cross-sections look fine:

In [ ]:
pl.figure(figsize=(12,8))
xx = np.linspace(0, 1, 100)
for y in np.linspace(0.1,0.9,5):
    pl.plot(xx, [lag([x, y]) for x in xx], label="$y=%.1f$" % y)
_ = pl.legend(fancybox=True, framealpha=0.8)

If we compare these solutions to the one with Hermite elements, not so much:

In [ ]:
pl.figure(figsize=(12,8))
xx = np.linspace(0,1,100)
pl.plot(xx, [lag([x, x]) for x in xx], label="$u_l(x,x)$")
pl.plot(xx, [her([x, x]) for x in xx], label="$u_h(x,x)$")
_ = pl.legend(fancybox=True, framealpha=0.8)

The (normalized) distance between the Lagrange and Hermite solutions is huge: 3% in $L^2$ norm and 16% in $H^1$ norm for a 10x10 mesh. However in a 20x20 the errors go down by almost an order of magnitude to 0.3% and 3.8% respectively. Is all this simply a consequence of the convergence properties of Hermite elements?

In [ ]:
diff = project(lag - her, V)
#diff.vector().abs()
i = 2
out =  '\\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{L_2}}{||u_{\\text{l%d}}||_{L_2}} \\approx %.4f,\ \ \ '\
       ' \\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{H_1}}{||u_{\\text{l%d}}||_{H_1}} \\approx %.4f \\\~ '\
       % (i, i, norm(diff, 'L2') / norm(lag, 'L2'), i, i, norm(diff, 'H1') / norm(lag, 'H1'))
from IPython.display import Math
Math(out)

**Nevertheless:** The fact that the difference is clearly skewed along one direction seems to indicate some problem with the implementation

In [ ]:
pl.figure(figsize=(10,10))
_ = plot(diff, title="$u_{l%d} - u_{h3}$" % i, cmap='bone')

We can plot the difference along the axes $x=y$, $x=1-y$:

In [ ]:
pl.figure(figsize=(12,8))
C = norm(lag.vector(), norm_type='linf')
pl.plot(xx, [diff(x, x)/C for x in xx], label="$(u_{l2} - u_{h3})(x,x)$")
pl.plot(xx, [diff(x, 1-x)/C for x in xx], label="$(u_{l2} - u_{h3})(x,1-x)$")
pl.title("Difference, normalized by $||u_{l2}||_{L^\infty{}}$")
_ = pl.legend(loc='lower right')

# Boilerplate

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: block;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")